# Using Tools


In [25]:
import instructor
from openai import OpenAI

#Only for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

#--------------------------------------------------------
# Initialize the OpenAI client for Ollama   
# And add instructor wrapper for Pydantic validation
#--------------------------------------------------------
client = OpenAI(
    base_url="http://localhost:11434/v1/", 
    api_key=""
)
client = instructor.patch(client)

In [31]:
import requests
from pydantic import BaseModel, Field
import json

#----------------------------------------------------------
# Tool 
#----------------------------------------------------------
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

#--------------------------------------------------------
# Tool response model 
#--------------------------------------------------------
class WeatherResponse(BaseModel):
    temperature: float = Field(description="The current temperature in celsius for the given location.")
    response: str = Field(description="A natural language response to the user's question.")


#--------------------------------------------------------
# Tool schema
#--------------------------------------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [ ]:
#---------------------------------------------------------------------------------------------
# 這是 OpenAI API 官方標準的端點 ===> 用於創建聊天對話的補全 (chat completion)。
#---------------------------------------------------------------------------------------------
# 1. 根據 messages 和 tools，模型決定是要直接生成文字回覆，還是要呼叫工具 (tool call)。
#    當模型決定呼叫工具時，它的回覆會包含工具名稱和參數，但不會執行工具。
#---------------------------------------------------------------------------------------------
completion2 = client.chat.completions.create(
    model="llama3.1:8b",    
    messages=[            
        {"role": "system",  "content": "You are a helpful weather assistant."},
        {"role": "user",    "content": "What's the weather like now in Tainan, Taiwan now ?"},
    ],     
    temperature=0, 
    tools=tools,      
)

print(f"\nchat completion ===>\n", json.dumps(completion2.model_dump(), indent=2))   

#----------------------------------------------------------------------------
# 2. 如果 LLM 說要 Call Tool
#----------------------------------------------------------------------------
tool_calls = completion2.choices[0].message.tool_calls
if tool_calls:  
    for tool_call in completion2.choices[0].message.tool_calls:
        # 3. 取出 Tool 的 name 和 參數    
        name = tool_call.function.name    
        args = json.loads(tool_call.function.arguments)        
        #print(f"\nCalling function: {name} with args: {args}")               
        #print(f"\nOriginal human message:", messages)    
            
        # 4. 把模型的回覆加入到 messages 列表中。使 messages 現在包含了使用者發問和模型的回覆     
        messages.append(completion2.choices[0].message)    
        print(f"\nMessages after create: ===>\n", completion2.choices[0].message)     
              
        # 5. Tool Calling
        result = None
        if name == "get_weather":
            result = get_weather(**args)           
            messages.append(
                {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
            )  
            print(f"\nTool Calling 結果 ===>\n {result}")           
            

In [33]:
#----------------------------------------------------------------------------
# 6. 最後讓 LLM 根據工具回傳的內容，生成你想要的自然語言回應。
#    而且回傳格式會符合你定義的 WeatherResponse Pydantic schema
#----------------------------------------------------------------------------
completion = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)

final_response = completion.choices[0].message.parsed
print("\nWeatherResponse Model 定義的 temperature ===> ", final_response.temperature)
print("WeatherResponse Model 定義的 Response ===> ",final_response.response)



WeatherResponse Model 定義的 temperature ===>  30.9
WeatherResponse Model 定義的 Response ===>  The current weather in Tainan, Taiwan is hot with a temperature of 30.9°C (87.6°F), wind speed around 11.8m/s and it will decrease over the time by around 1-2°C at about 11:15 AM with light winds


🧑‍💻 使用者提問
     ↓
🧠 Step 1: client.chat.completions.create(...)
     └─ LLM 判斷需要用 tool，例如 get_weather
     └─ 產生 tool_call 指令 + arguments
     ↓
📨 Step 2: ✨ messages.append(模型提出的 tool_call message)
     ↓
⚙️ Step 3: 程式實際執行對應的 Tool function（如 get_weather(**args)）
     ↓
📨 Step 4: ✨ messages.append({
         role: "tool",
         tool_call_id: tool_call.id,
         content: 工具回傳結果（如天氣資料）
     })
     ↓
🧠 Step 5: client.beta.chat.completions.parse(...)
     └─ 模型根據完整上下文（使用者提問 + tool_call + 工具結果）
     └─ 產生最終自然語言回答 & 結構化資料（符合 Pydantic schema）
